In [14]:
import openai
from openai import OpenAI
client = OpenAI(api_key="Your_API_Key")
import re
import json
import pandas as pd
import numpy as np
from statistics import mode

In [7]:
df = pd.read_hdf("/content/drive/MyDrive/mind_inference_experiment/result_users_ebdL.h5", key='user')
df

,Index,Category,Product,Comment,Post_or_Pre,Do-goals,Motor-goals,Be-goals,FPN1_Autonomy,FPN2_Beauty,...,FPN11_Relatedness,FPN12_Security,FPN13_Stimulation,fpns,pr_dg,pr_mg,pr_bg,dg_embedding,mg_embedding,bg_embedding
0,0,2,Luggage,The luggage is small in size to fit as a cabbi...,post,I aim to carry my materials compactly and comf...,"The bag should be an adequate size, big enough...",I hope to be relaxed and in a peaceful state w...,4,4,...,1,5,1,"[4, 4, 5, 3, 3, 1, 3, 1, 5, 2, 1, 5, 1]",I aim to carry my materials compactly and comf...,"The bag should be an adequate size, big enough...",I hope to be relaxed and in a peaceful state w...,"[-0.024392396211624146, 0.027402115985751152, ...","[-0.048109713941812515, 0.01738385297358036, 0...","[-0.03234438970685005, 0.020991625264286995, -..."
1,1,2,travel adapter (American to European wall outl...,It is small and lightweight which made it good...,post,I aim to use this adaptor to charge my laptop ...,"By plugging the adaptor into the wall, my lapt...",I aspire to conduct engineering research in Eu...,5,1,...,1,4,5,"[5, 1, 1, 1, 5, 1, 4, 1, 4, 1, 1, 4, 5]",I aim to use this adapter to charge my laptop ...,"By plugging the adapter into the wall, my lapt...",I aspire to conduct engineering research in Eu...,"[-0.02032318338751793, 0.04584910348057747, -0...","[0.0001383212220389396, 0.04337985813617706, -...","[-0.03136971965432167, 0.03054780513048172, -0..."
2,2,2,Portable charger,"This portable charger is the perfect size, and...",post,I am to recharge my phone to 100% with this po...,By plugging in a usb/usb-c charger into this p...,I aspire to be in control and feel secure with...,4,1,...,1,5,2,"[4, 1, 5, 1, 2, 2, 1, 1, 4, 1, 1, 5, 2]",I am going to recharge my phone to 100 percent...,By plugging in a USB/USB-C charger into this p...,I aspire to be in control and feel secure with...,"[-0.045712582767009735, 0.056636933237314224, ...","[-0.03238840773701668, 0.0747632384300232, -0....","[-0.0349658764898777, 0.04574235528707504, -0...."
3,3,3,desktop computer (Mac Mini),"The set up was easy and straightforward, and t...",post,I aim to complete multiple tasks quickly on th...,I can complete multiple tasks on the desktop b...,I aspire to be more efficient at my work,3,2,...,1,1,1,"[3, 2, 5, 1, 5, 1, 3, 1, 1, 1, 1, 1, 1]",I aim to complete multiple tasks quickly on th...,I can complete multiple tasks on the desktop b...,I aspire to be more efficient at my work.,"[-0.022806786000728607, 0.002680284669622779, ...","[0.004365907981991768, 0.03254324570298195, -0...","[-0.02478894405066967, 0.014668968506157398, -..."
4,4,3,laptop,The laptop is very sleek and had a long batter...,post,I aim to use the laptop's apps and browser for...,"I open the laptop, open VSCode, and start typi...",I aspire to increase my knowledge and improve ...,2,1,...,1,2,1,"[2, 1, 1, 1, 5, 1, 5, 2, 4, 3, 1, 2, 1]",I aim to use the laptop's applications and web...,"I open the laptop, open Visual Studio Code, an...",I aspire to increase my knowledge and improve ...,"[-0.025130746886134148, -0.0015777747612446547...","[0.030043108388781548, 0.024901222437620163, -...","[-0.036632269620895386, 0.015062123537063599, ..."
5,5,1,Sponge,"The sponge is functional enough when washing, ...",post,The goal is to ensure dishes are clean of food...,I should be able to apply water (warm) and dis...,"I hope to have a clean/hygenic environment, es...",2,2,...,1,4,1,"[2, 2, 4, 1, 3, 4, 1, 1, 1, 1, 1, 4, 1]",The goal is to ensure dishes are clean of food...,I should be able to apply warm water and dish ...,I hope to have a clean and hygienic environmen...,"[-0.021783504635095596, -0.021083584055304527,...","[-0.019238632172346115, -0.03926096484065056, ...","[-0.02034141682088375, 0.005699168425053358, -..."
6,6,2,Portable charger,"I like this charger, it has a long battery lif...",post,I aim to charge my phone on the go. I aim to b...,I need to make sure my portable charger is ful...,I aspire to stay connected; I need to feel sa

In [8]:
def gpt(prompt: str, model='gpt-4-0613', temperature = 0.7, n=5):
    output = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=1000,
        n=n
    )
    choices = []
    choices.extend([choice.message.content for choice in output.choices])
    return choices

def vote(instruction: str, choices: list, model='gpt-4-0613', temperature = 0.7, n=5):
    choices_prompt = ''
    for i, y in enumerate(choices, 1):
        choices_prompt += f'Choice {i}:\n{y}\n'

    prompt_vote = f'''Given an instruction and several choices, decide which choice is most promising.
    <Instruction>
    {instruction}

    <Choices>
    {choices_prompt}

    Analyze each choice in detail, then conclude your answer in the last line.
    Your concludsion should strictly follow the format: "The best choice is {{s}}", where s the integer id of the choice.

    '''
    vote_outputs = gpt(prompt_vote, model=model, temperature=temperature, n=n)
    votes_values = vote_outputs_unwrap(vote_outputs, n)
    idx = votes_values.index(max(votes_values))
    chosen = choices[idx]
    vote_rate = max(votes_values)/n
    return (idx, chosen, vote_rate)

def vote_outputs_unwrap(vote_outputs: list, n_candidates: int):
    vote_results = [0] * n_candidates
    for vote_output in vote_outputs:
        pattern = r".*best choice is .*(\d+).*"
        match = re.match(pattern, vote_output, re.DOTALL)
        if match:
            vote = int(match.groups()[0]) - 1
            if vote in range(n_candidates):
                vote_results[vote] += 1
        else:
            print(f'vote no match: {[vote_output]}')
    return vote_results

def get_most_frequent(lists):
    transposed = zip(*lists)
    modes = [mode(group) for group in transposed]
    return modes

In [9]:
def perspective_taking_prompt(user_input):

    prompt_pt = f'''You will be provided with a user comment regarding a {user_input[1]}.
    Considering the given information, you are asked to take the perspective of the user and imagine the complete context related to the his/her experience with the product.

    <comment>
    {user_input[0]}

    <perspective taking>
    '''
    return prompt_pt

def goal_inference_prompt(user_input, perspective_taking, goal_output_format):

    prompt_infer = f'''You will be provided with a user comment regarding a {user_input[1]}.
    In addition, a description of the context about the user's experience will be provided.
    Based on the comment and context, your task is to take the user's perspective and infer his/her underlying goals in using the product.
    Please come up with at least two goals for each goal-inference question, and you are encouraged to infer as many goals as possible.
    Make sure to only report goals related to the product and its use. Please do not include any goals related to purchasing decisions, cost considerations, or general consumer experiences.

    Question 1：
    Taking the user's perspective, what are the specific tasks or actions that you aim to achieve or complete?
        Example format: I aim to make a telephone call; I aim to drive my car for the daily commute.

    Question 2:
    Taking the user's perspective, what are the concrete sub-actions or steps you need to take to achieve the tasks/actions you listed above?
        Example format: By pressing the right buttons, I should be able to dial the number I intend to call; By turning the ignition, adjusting mirrors, and following my routine route, I can engage in the process of driving.

    Question 3:
    Taking the user's perspective, what are your overarching goals that are related to your broader life aspirations, self-perception, and/or desires about your own identity?
        Example format: I aspire to be close to others; I aspire to feel secure and have a controllable space.

    Analyze question by question, and then conclude your answers at the end in JSON format as in:
    {goal_output_format}

    <Comment>
    {user_input[0]}

    <Context>
    {perspective_taking}
    '''
    return prompt_infer

def need_attribution_prompt(fn_material, user_input, perspective_taking, be_goal, need_output_format):
    prompt_attribution = f'''Please first familiarize yourself with the 13 fundamental psychology needs below:
    {fn_material}

    Now, you will be provided with a user comment regarding a {user_input[1]}.
    In addition, you will also be provied with a description of the context about the user's experience and the user's overarching goals that are related to their broader life aspirations, self-perception, and/or desires about their own identity.
    Your task is to attribute the user's goals to each of the fundamental psychological need using a 5-point Likert scale, where 1 = Not attributed at all, 2 = Slightly attributed, 3 = Somewhat attributed, 4 = Attributed, and 5 = Highly attributed.
    Analyze the attribution of each psychological need in details, and then conclude your answers at the end in JSON format as in:
    {need_output_format}

    <Comment>
    {user_input[0]}

    <Context>
    {perspective_taking}

    <Goals>
    {be_goal}
    '''
    return prompt_attribution

In [10]:
goal_output_format = '''
{
    "Question 1": "...",
    "Question 2": "...",
    "Question 3": "..."
}
'''

need_output_format = '''
{
    "autonomy": "1",
    "beauty": "1",
    "comfort": "2",
    "community": "1",
    "competence": "4",
    "fitness": "2",
    "impact": "1",
    "morality": "1",
    "purpose": "2",
    "recognition": "5",
    "relatedness": "1",
    "security": "2",
    "stimulation": "4"
}
'''

def extract_json(text):
    regex = r'\{.*?\}'
    matches = re.findall(regex, text, re.DOTALL)
    json_objects = []
    for match in matches:
        try:
            json_object = json.loads(match)
            json_objects.append(json_object)
        except json.JSONDecodeError:
            pass

    return json_objects

def extract_values_from_dicts(dicts):
    values_list = []
    for d in dicts:
        for value in d.values():
            values_list.append(value)
    return values_list



In [11]:
def mind_inference_tot(user_input: tuple, # user_input: (comment, product_type)
                       fn_material_path = '/content/drive/MyDrive/mind_inference_experiment/13 FNs.txt',
                       goal_output_format = goal_output_format,
                       need_output_format = need_output_format,
                       model='gpt-4-0613',
                       temperature=0.7,
                       n_generate=5,
                       n_vote=5):
    ### CoT: n_generate = 1
    ### set temperature = 0, n_generate = 1, n_vote = 1 for the CoT experiment in the paper
    ### ToT: temperature > 0, n_generate >1, n_vote > 1
    ### set temperature = 0.7, n_generate = 5, n_vote = 5 for the ToT experiment in the paper, note that ToT results are not repoduciable due to high temperature

    if n_generate >= 1 and n_generate <= 10:
        pass
    else:
        print('n_generate out of range [1, 10]')
        return None

    do_tot = n_generate > 1 and n_vote > 1

    # perspective-taking
    pt_prompt = perspective_taking_prompt(user_input)
    pt_choices = gpt(prompt=pt_prompt, model=model, temperature=temperature, n=n_generate)
    if do_tot:
        pt_idx, pt_selected, pt_rate = vote(instruction=pt_prompt, choices=pt_choices, model=model, temperature=temperature, n=n_vote)
    else:
        pt_selected = pt_choices[0]

    # goals inference
    goal_prompt = goal_inference_prompt(user_input, pt_selected, goal_output_format)
    goal_choices = gpt(prompt=goal_prompt, model=model, temperature=temperature, n=n_generate)
    if do_tot:
        goal_idx, goal_selected, goal_rate = vote(instruction=goal_prompt, choices=goal_choices, model=model, temperature=temperature, n=n_vote)
    else:
        goal_selected = goal_choices[0]
    goals = extract_json(goal_selected)
    goals_result = extract_values_from_dicts(goals)

    # needs attribution
    fns = open(fn_material_path,'r')
    fns_prompt = fns.read()
    need_prompt = need_attribution_prompt(fns_prompt, user_input, pt_selected, goals_result[2], need_output_format)
    need_choices = gpt(prompt=need_prompt, model=model, temperature=temperature, n=n_generate)
    if do_tot:
        needs = [extract_values_from_dicts(extract_json(choice)) for choice in need_choices]
        needs_result = get_most_frequent(needs)
    else:
        needs_result = extract_values_from_dicts(extract_json(need_choices[0]))

    return pt_selected, goals_result, needs_result

In [12]:
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((np.array(vec1) - np.array(vec2)) ** 2))

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def normalize_needs_vector(elements):
    normalized_list = []
    for element in elements:
        int_element = int(element)
        normalized_element = (int_element - 1) / 4
        normalized_list.append(normalized_element)
    return normalized_list

In [ ]:
### example CoT inference with GPT-4
df_cot4 = pd.DataFrame(columns=['index','perspective_taking','dg','mg','bg','fpns','dg_embedding','mg_embedding','bg_embedding',"dg_cos","mg_cos","bg_cos","fpn_cos","fpn_euc"])

for i in range(len(df)):
    df_cot4.loc[i,'index'] = df.loc[i,'Index']
    user_input = tuple(df.loc[i, ['Comment','Product']])
    pt, goals, needs = mind_inference_tot(user_input, model='gpt-4-0613', temperature=0, n_generate=1, n_vote=1)
    ## for ToT set parameters so that: temperature > 0, n_generate >1, n_vote > 1, note that ToT consume tokens fast and can be expensive to run using GPT-4
    for a in range(len(goals)):
      goal = goals[a]
      if type(goal) == list:
        goal = '; '.join(goal)
        goals[a] = goal
    ### get goal embeddings
    ebd_dg, ebd_mg, ebd_bg = get_embedding(goals[0]), get_embedding(goals[1]), get_embedding(goals[2])
    df_cot4.loc[i,'perspective_taking'] = pt
    df_cot4.loc[i,['dg','mg','bg']] = goals
    df_cot4.loc[i,'fpns'] = needs
    df_cot4.loc[i,['dg_embedding','mg_embedding','bg_embedding']] = ebd_dg, ebd_mg, ebd_bg
    ### measure similarity scores
    base_dg, base_mg, base_bg, base_fpns = df.loc[i, ['dg_embedding','mg_embedding','bg_embedding','fpns']]
    cos_d, cos_m, cos_b = cosine_similarity(base_dg,ebd_dg), cosine_similarity(base_mg,ebd_mg), cosine_similarity(base_bg,ebd_bg)
    cos_fpn = cosine_similarity(normalize_needs_vector(base_fpns),normalize_needs_vector(needs))
    euc_fpn = euclidean_distance(normalize_needs_vector(base_fpns),normalize_needs_vector(needs))
    df_cot4.loc[i, ["dg_cos","mg_cos","bg_cos","fpn_cos","fpn_euc"]] = cos_d, cos_m, cos_b, cos_fpn, euc_fpn

In [16]:
df_cot4.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_cot_gpt4_scores.h5', key='gpt')

<ipython-input-16-4dd70c67e14c>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['index', 'perspective_taking', 'dg', 'mg', 'bg', 'fpns', 'dg_embedding',
       'mg_embedding', 'bg_embedding', 'dg_cos', 'mg_cos', 'bg_cos', 'fpn_cos',
       'fpn_euc'],
      dtype='object')]

  df_cot4.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_cot_gpt4_scores.h5', key='gpt')
